## Video Games Sales

`pip install pandas`

In [2]:
import pandas as pd

In [17]:
# Load the datasets with index
resources_path = "resources/"

resources = {
    "vg_sales": "video_games_sales_as_at_22_dec_2016.csv",
    "vg_developers": "video_games_developers.csv"
}

# vg_sales
vg_sales = pd.read_csv(resources_path + resources["vg_sales"], index_col=0)

# vg_developers
vg_developers = pd.read_csv(resources_path + resources["vg_developers"], index_col=0)
vg_developers = vg_developers.iloc[:, [0, 2, 3]]
vg_developers.rename(columns={'Est.': 'Year_of_Establishment'}, inplace=True)

In [18]:
# columns
print("vg_sales.columns :", vg_sales.columns)
print("vg_developers.columns :", vg_developers.columns)

vg_sales.columns : Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')
vg_developers.columns : Index(['City', 'Country', 'Year_of_Establishment'], dtype='object')
